# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv('weather_data.csv')
weather_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,barrow,71.2906,-156.7887,2.01,93,100,6.69,US,1658877943
1,1,talaya,61.3833,152.7500,12.11,84,100,2.40,RU,1658877944
2,2,coquimbo,-29.9533,-71.3436,12.85,94,20,2.57,CL,1658877672
3,3,cape town,-33.9258,18.4232,12.29,91,40,1.03,ZA,1658877741
4,4,edson,53.5835,-116.4356,26.08,34,40,2.06,CA,1658877944
5,5,ilulissat,69.2167,-51.1000,7.01,93,100,3.09,GL,1658877945
6,6,mayo,38.8876,-76.5119,26.23,83,100,2.06,US,1658877945
7,7,katsuura,35.1333,140.3000,27.44,85,77,6.56,JP,1658877945
8,8,rocha,-34.4833,-54.3333,16.21,96,98,2.16,UY,1658877946
9,9,constitucion,-35.3333,-72.4167,11.16,93,94,2.85,CL,1658877946


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Bring in location data pairs, humidity data, define map and layer, and add layer to map
locations = weather_df[["Lat", "Lng"]].astype(float)
weight = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Ideal conditions: max temperature between 20 and 30 celsius,
# humidity less than 80, cloudiness less than 30

upper_temp = weather_df[weather_df['Max Temp'] <= 30]
ideal_temp = upper_temp[upper_temp['Max Temp'] >= 20]

ideal_humidity = ideal_temp[ideal_temp['Humidity'] <= 80]

ideal_df = ideal_humidity[ideal_humidity['Cloudiness'] <= 30]

ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,jamestown,42.0970,-79.2353,23.92,54,0,4.12,US,1658877948
19,19,bethel,41.3712,-73.4140,27.05,53,0,3.09,US,1658877754
39,39,quetta,30.1990,67.0097,23.05,72,15,1.74,PK,1658877955
50,50,atuona,-9.8000,-139.0333,25.11,75,10,7.41,PF,1658877958
70,70,suntar,62.1444,117.6319,25.68,65,1,2.13,RU,1658877964
...,...,...,...,...,...,...,...,...,...,...
527,527,jardim,-21.4803,-56.1381,25.15,32,0,1.80,BR,1658878100
530,530,naze,28.3667,129.4833,27.51,74,10,2.67,JP,1658878100
535,535,eyl,7.9803,49.8164,24.68,74,4,10.57,SO,1658878102
541,541,roma,43.2128,-75.4557,23.86,59,0,3.58,US,1658878103


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create dataframe and empty column
hotel_df = pd.DataFrame({
    "City": ideal_df['City'],
    "Country": ideal_df['Country'],
    "Lat": ideal_df['Lat'],
    "Lng": ideal_df['Lng']
})

hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
15,jamestown,US,42.0970,-79.2353,
19,bethel,US,41.3712,-73.4140,
39,quetta,PK,30.1990,67.0097,
50,atuona,PF,-9.8000,-139.0333,
70,suntar,RU,62.1444,117.6319,


In [13]:
# Loop through dataframe and add nearby hotel results
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# iterate through df
for index, row in hotel_df.iterrows():
    params['location'] = f'{row["Lat"]}, {row["Lng"]}'
    response = requests.get(base_url, params=params).json()

    # print(response)

    try:
        results = response
        print(f"the closest hotel is {results['results'][0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results['results'][0]['name']
        # hotel_df.loc[index, 'Country'] = results['results'][0]['country']

    except (KeyError, IndexError):
        print("=skipping missing field/result")

    print('------')

the closest hotel is Hampton Inn & Suites Jamestown.
------
the closest hotel is Courtyard by Marriott Danbury.
------
the closest hotel is Hotel Deluxe.
------
the closest hotel is Villa Enata.
------
the closest hotel is Гостевая квартира.
------
the closest hotel is Gostinichnyy Kompleks Aylgy.
------
the closest hotel is Ephira Hotel.
------
the closest hotel is The Explorer Hotel.
------
the closest hotel is Floresta Amazonica Hotel.
------
the closest hotel is Chernyseyskiy.
------
the closest hotel is Arman hotel.
------
the closest hotel is Veranda Paul et Virginie Hotel & Spa.
------
=skipping missing field/result
------
the closest hotel is Helios Hotel.
------
=skipping missing field/result
------
the closest hotel is Paradise Inn.
------
the closest hotel is Apel'sin.
------
the closest hotel is Gulisitan Hotel.
------
the closest hotel is Lovers Point Inn.
------
=skipping missing field/result
------
the closest hotel is Agan-Hotel.
------
the closest hotel is Гостиница "C

In [14]:
hotel_df.head(10)

,City,Country,Lat,Lng,Hotel Name
15,jamestown,US,42.0970,-79.2353,Hampton Inn & Suites Jamestown
19,bethel,US,41.3712,-73.4140,Courtyard by Marriott Danbury
39,quetta,PK,30.1990,67.0097,Hotel Deluxe
50,atuona,PF,-9.8000,-139.0333,Villa Enata
70,suntar,RU,62.1444,117.6319,Гостевая квартира
74,zhigansk,RU,66.7697,123.3711,Gostinichnyy Kompleks Aylgy
81,corinto,GR,37.9407,22.9573,Ephira Hotel
86,yellowknife,CA,62.4560,-114.3525,The Explorer Hotel
94,alta floresta,BR,-9.8756,-56.0861,Floresta Amazonica Hotel
101,chernyshevskiy,RU,63.0128,112.4714,Chernyseyskiy


In [15]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)


# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))